In [59]:
import os
from pytse_client.download import download_financial_indexes
import pandas as pd
import jalali_pandas
import math
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [66]:
def return_calculator(df):
    period_return = 1
    li = list(df)
    for day, _ in df.iteritems():
        idx = df.index.get_loc(day)
        elem = li[idx]
        if (math.isnan(elem)):
            elem = outlier_handler(elem)
            if elem == False:
                continue
        period_return = period_return * (1+elem)
    period_return = period_return - 1
    return period_return

def outlier_handler(ret):
    if (math.isnan(ret)):
        return False

In [63]:
desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
bourse_view = os.path.join(desktop, "sector_slc", "bourse_view")
market_data_processed_path = os.path.join(bourse_view, "market_data_processed")
sectors_data_processed_path = os.path.join(bourse_view, "sectors_data_processed")
list_of_sector_files = os.listdir(sectors_data_processed_path)
list_of_sector_files = list_of_sector_files[1:]
list_of_market_files = os.listdir(market_data_processed_path)
list_of_market_files = list_of_market_files[1:]

In [90]:
file_name = list_of_sector_files[11]
data = pd.read_excel(sectors_data_processed_path+ "\\" + file_name)
data = data.fillna(method='ffill')

first_year_quarter = data.quarter.iloc[0]
first_year = data.year.iloc[0]
last_year_quarter = data.quarter.iloc[-1]
last_year = data.year.iloc[-1]

empty_li_quarters = []
empty_li_year = []

for year in range(first_year, last_year + 1):

    if year == first_year:
        empty_li_quarters += [i for i in range(first_year_quarter,5)]
        empty_li_year += [year] * (5-first_year_quarter)

    elif year == last_year:
        empty_li_quarters += [i for i in range(1,last_year_quarter+1)]
        empty_li_year += [year] * (last_year_quarter)

    else:
        empty_li_quarters += [i for i in range(1,5)]
        empty_li_year += [year] * 4

quarterly_data = pd.DataFrame(columns=["year","quarter","quarterly_return","quarterly_trade_value","P/E-ttm"])
quarterly_data["year"] = empty_li_year
quarterly_data["quarter"] = empty_li_quarters

for year in range(first_year, last_year + 1):

    if year == first_year:
        for quarter in range(first_year_quarter, 5):
            idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
            ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
            pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
            trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
            quarterly_data["quarterly_return"][idx] = ret
            quarterly_data["quarterly_trade_value"][idx] = trade_value
            quarterly_data["P/E-ttm"][idx] = pe

    elif year == last_year:
        for quarter in range(1, last_year_quarter + 1):
            idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
            ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
            pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
            trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
            quarterly_data["quarterly_return"][idx] = ret
            quarterly_data["quarterly_trade_value"][idx] = trade_value
            quarterly_data["P/E-ttm"][idx] = pe
    else:
        for quarter in range(1, 5):
            idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
            ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
            pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
            trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
            quarterly_data["quarterly_return"][idx] = ret
            quarterly_data["quarterly_trade_value"][idx] = trade_value
            quarterly_data["P/E-ttm"][idx] = pe

quarterly_data["P/E-ttm_1_lag"] = quarterly_data["P/E-ttm"].shift(1)
quarterly_data["P/E-ttm_2_lag"] = quarterly_data["P/E-ttm"].shift(2)
quarterly_data["quarterly_return_1_lag"] = quarterly_data["quarterly_return"].shift(1)
quarterly_data["quarterly_return_2_lag"] = quarterly_data["quarterly_return"].shift(2)
quarterly_data = quarterly_data.dropna().reset_index().drop(["index"],axis=1)

In [126]:
data = pd.read_csv(market_data_processed_path+ "\\" + list_of_market_files[0])
data = data.drop(["Unnamed: 0"],axis=1)
data = data.fillna(method='ffill')
empty_li_quarters = []
empty_li_year = []

for year in range(first_year, last_year + 1):

    if year == first_year:
        empty_li_quarters += [i for i in range(first_year_quarter,5)]
        empty_li_year += [year] * (5-first_year_quarter)

    elif year == last_year:
        empty_li_quarters += [i for i in range(1,last_year_quarter+1)]
        empty_li_year += [year] * (last_year_quarter)

    else:
        empty_li_quarters += [i for i in range(1,5)]
        empty_li_year += [year] * 4

market_quarterly_data = pd.DataFrame(columns=["year","quarter","index_quarterly_return","usd_quarterly_return","quarterly_trade_value","market_pe"])
market_quarterly_data["year"] = empty_li_year
market_quarterly_data["quarter"] = empty_li_quarters

for year in range(first_year, last_year + 1):

    if year == first_year:
        for quarter in range(first_year_quarter, 5):
            idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
            index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
            usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
            pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
            trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
            market_quarterly_data["index_quarterly_return"][idx] = index_ret
            market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
            market_quarterly_data["quarterly_trade_value"][idx] = trade_value
            market_quarterly_data["market_pe"][idx] = pe

    elif year == last_year:
        for quarter in range(1, last_year_quarter + 1):
            idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
            index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
            usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
            pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
            trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
            market_quarterly_data["index_quarterly_return"][idx] = index_ret
            market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
            market_quarterly_data["quarterly_trade_value"][idx] = trade_value
            market_quarterly_data["market_pe"][idx] = pe
    else:
        for quarter in range(1, 5):
            idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
            index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
            usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
            pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
            trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
            market_quarterly_data["index_quarterly_return"][idx] = index_ret
            market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
            market_quarterly_data["quarterly_trade_value"][idx] = trade_value
            market_quarterly_data["market_pe"][idx] = pe

market_quarterly_data["index_quarterly_return_1_lag"] = market_quarterly_data["index_quarterly_return"].shift(1)
market_quarterly_data["index_quarterly_return_2_lag"] = market_quarterly_data["index_quarterly_return"].shift(2)
market_quarterly_data["usd_quarterly_return_1_lag"] = market_quarterly_data["usd_quarterly_return"].shift(1)
market_quarterly_data["usd_quarterly_return_2_lag"] = market_quarterly_data["usd_quarterly_return"].shift(2)
market_quarterly_data["market_pe_1_lag"] = market_quarterly_data["market_pe"].shift(1)
market_quarterly_data["market_pe_2_lag"] = market_quarterly_data["market_pe"].shift(2)
market_quarterly_data = market_quarterly_data.dropna().reset_index().drop(["index"],axis=1)

In [143]:
quarterly_data["quarterly_usd_return"] = market_quarterly_data["usd_quarterly_return"]
quarterly_data["quarterly_usd_return_1_lag"] = market_quarterly_data["usd_quarterly_return_1_lag"]
quarterly_data["quarterly_usd_return_2_lag"] = market_quarterly_data["usd_quarterly_return_2_lag"]
quarterly_data["market_pe"] = market_quarterly_data["market_pe"]
quarterly_data["market_pe_1_lag"] = market_quarterly_data["market_pe_1_lag"]
quarterly_data["market_pe_2_lag"] = market_quarterly_data["market_pe_2_lag"]
quarterly_data["relative_trade_value"] = 100 * quarterly_data["quarterly_trade_value"] / market_quarterly_data["quarterly_trade_value"]
quarterly_data["index_quarterly_return"] = market_quarterly_data["index_quarterly_return"]
quarterly_data["index_quarterly_return_1_lag"] = market_quarterly_data["index_quarterly_return_1_lag"]
quarterly_data["index_quarterly_return_2_lag"] = market_quarterly_data["index_quarterly_return_2_lag"]
quarterly_data["quarter_excess_return"] = quarterly_data["quarterly_return"] - quarterly_data["index_quarterly_return"]

In [144]:
quarterly_data

,year,quarter,quarterly_return,quarterly_trade_value,P/E-ttm,P/E-ttm_1_lag,P/E-ttm_2_lag,quarterly_return_1_lag,quarterly_return_2_lag,quarterly_usd_return,quarterly_usd_return_1_lag,quarterly_usd_return_2_lag,market_pe_1_lag,market_pe_2_lag,market_pe,relative_trade_value,index_quarterly_return,index_quarterly_return_1_lag,index_quarterly_return_2_lag,quarter_excess_return
0,1391,3,0.437708,3362069715.490431,7.383738,6.708726,8.553946,0.113784,0.081,0.270593,0.328228,-0.045431,4.603397,5.742991,5.573097,0.249776,0.325853,0.008189,0.007921,0.111854
1,1391,4,-0.265165,468728704.204334,7.603099,7.383738,6.708726,0.437708,0.113784,0.103728,0.270593,0.328228,5.573097,4.603397,5.670137,0.027859,0.081995,0.325853,0.008189,-0.34716
2,1392,1,0.218553,1578517862.426773,6.521231,7.603099,7.383738,-0.265165,0.437708,-0.001468,0.103728,0.270593,5.670137,5.573097,6.123779,0.092959,0.281163,0.081995,0.325853,-0.06261
3,1392,2,0.172258,2930435978.839247,6.947317,6.521231,7.603099,0.218553,-0.265165,-0.138235,-0.001468,0.103728,6.123779,5.670137,6.540758,0.115143,0.247695,0.281163,0.081995,-0.075437
4,1392,3,1.135938,9892618558.911112,14.182752,6.947317,6.521231,0.172258,0.218553,0.005119,-0.138235,-0.001468,6.540758,6.123779,8.192748,0.20019,0.394791,0.247695,0.281163,0.741148
5,1392,4,-0.114146,15192802090.402132,12.402353,14.182752,6.947317,1.135938,0.172258,0.02343,0.005119,-0.138235,8.192748,6.540758,7.193751,0.427106,-0.092053,0.394791,0.247695,-0.022093
6,1393,1,0.200698,22257931714.645145,17.840797,12.402353,14.182752,-0.114146,1.135938,0.064035,0.02343,0.005119,7.193751,8.192748,6.546739,1.166536,-0.069841,-0.092053,0.394791,0.270539
7,1393,2,-0.122578,6845157948.492406,14.338957,17.840797,12.402353,0.200698,-0.114146,-0.011225,0.064035,0.02343,6.546739,7.193751,5.61127,0.564705,0.01707,-0.069841,-0.092053,-0.139648
8,1393,3,-0.094699,16500957029.944443,14.977823,14.338957,17.840797,-0.122578,0.200698,0.106906,-0.011225,0.064035,5.61127,6.546739,5.466739,0.803169,-0.027087,0.01707,-0.069841,-0.067612
9,1393,4,-0.188472,2631714121.088624,13.545531,14.977823,14.338957,-0.094699,-0.122578,-0.045869,0.106906,-0.011225,5.466739,5.61127,5.009215,0.236981,-0.109201,-0.027087,0.01707,-0.079271
